In [1]:
from qft import *
from qiskit import qasm2
import pandas as pd

In [2]:
num_q = 5
cir = QFT(num_q)
cir.draw()

                                                                 
q_0: ─■──■──■──■──■─────■─────■─────■────────────────────────────
      │  │  │  │  │     │     │     │                            
q_1: ─■──■──┼──┼──┼──■──┼──■──┼──■──┼──■──■─────■────────────────
            │  │  │  │  │  │  │  │  │  │  │     │                
q_2: ───────■──■──┼──■──┼──■──┼──┼──┼──┼──┼──■──┼──■──■──■───────
                  │     │     │  │  │  │  │  │  │  │  │  │       
q_3: ─────────────■─────■─────┼──■──┼──■──┼──■──┼──■──┼──┼──■──■─
                              │     │     │     │     │  │  │  │ 
q_4: ─────────────────────────■─────■─────■─────■─────■──■──■──■─
                                                                 


In [3]:
print(cir.num_gate)
sum([len(gates) for gates in cir.gate_list])

20


20

In [4]:
def move_time(distance):
    return 200*((distance/110)**(1/2))

In [5]:
import math
def set_parameters(default):
    para = {}
    if default:
        para['T_cz'] = 0.2  #us
        para['T_eff'] = 1.5e6 #us
        para['T_trans'] = 20 # us
        para['AOD_width'] = 3 #um
        para['AOD_height'] = 3 #um
        para['Move_speed'] = 0.55 #um/us
        para['F_cz'] = 0.995
        para['F_transfer'] = 0.999
        para['F_cz_idle'] = 1 - (1-para['F_cz'])/2
        para['R_b'] = 6 #um

    return para
def DasAtom_fidelity(parallel_gates, all_movements, num_q, gate_num):
    para = set_parameters(True)
    t_total = 0
    t_total += (len(parallel_gates) * para['T_cz']) # cz execution time, parallel
    t_move = 0
    for move in all_movements:
        t_total += (4 * para['T_trans']) # pick/drop/pick/drop
        t_move += (4 * para['T_trans'])
        max_dis = 0
        for each_move in move:
            x1, y1 = each_move[1][0],each_move[1][1]
            x2, y2 = each_move[2][0],each_move[2][1]
            dis = (abs(x2-x1)*para['AOD_width'])**2 + (abs(y2-y1)*para['AOD_height'])**2
            if dis > max_dis:
                max_dis = dis
        max_dis = math.sqrt(max_dis)
        t_total += (max_dis/para['Move_speed'])
        t_move += (max_dis/para['Move_speed'])

    t_idle = num_q * t_total - gate_num * para['T_cz']
    Fidelity = math.exp(-t_idle/para['T_eff']) * (para['F_cz']**gate_num)
    move_fidelity = math.exp(-t_move/para['T_eff'])
    return t_idle, Fidelity, move_fidelity

def Enola_fidelity(parallel_gates, all_movements, num_q, gate_num):
    para = set_parameters(True)
    cir_fidelity_1q_gate = 1
    cir_fidelity_2q_gate = 1
    cir_fidelity_2q_gate_for_idle = 1
    cir_fidelity_atom_transfer = 1
    cir_fidelity_coherence = 1
    cir_qubit_idle_time = [0 for _ in range(num_q)]

    # num_gate = sum([len(gates) for gates in parallel_gates])
    cir_fidelity_2q_gate = para['F_cz'] ** gate_num
    cir_fidelity_2q_gate_for_idle = para['F_cz_idle'] ** (2*(len(parallel_gates)-1)*num_q - 2*gate_num)

    for movements in all_movements:
        for mov in movements:
            q = mov[0]
            cir_qubit_idle_time[q] += 4*para['T_trans']
        cir_fidelity_atom_transfer *= para['F_transfer']**(4*len(mov))
        # TODO: only for d = 1
        for i in range(num_q):
            cir_qubit_idle_time[i] += move_time(para['R_b'])
    for t in cir_qubit_idle_time:
            cir_fidelity_coherence *= (1 - t/para["T_eff"])
    res = {"cir_fidelity_1q_gate":cir_fidelity_1q_gate,
            "cir_fidelity_2q_gate": cir_fidelity_2q_gate,
            "cir_fidelity_2q_gate_for_idle":cir_fidelity_2q_gate_for_idle,
            "cir_fidelity_atom_transfer":cir_fidelity_atom_transfer,
            "cir_fidelity_coherence":cir_fidelity_coherence,}
    cir_fidelity = cir_fidelity_1q_gate * cir_fidelity_2q_gate * cir_fidelity_2q_gate_for_idle \
        * cir_fidelity_atom_transfer * cir_fidelity_coherence
    res["cir_fidelity"] = cir_fidelity
    return res


In [6]:
def get_maps(cir) -> list[list[int]]:
    maps = []
    new_map = [[1,i+1] for i in range(cir.num_qubits)]
    AOD_qubits = []

    def update_AOD_qubits(locations, AOD_qubits,dir=[0,1]):
        new_loc = copy.deepcopy(locations)
        for qubit in AOD_qubits:
            new_loc[qubit][0] += dir[0]
            new_loc[qubit][1] += dir[1]
        return new_loc
            
    maps.append(new_map)
    for i in range(len(cir.gate_list)-1):
        # if i % 2 != 0:
        if i%2 == 0:
            AOD_qubits.append(i//2)
        new_map = update_AOD_qubits(new_map,AOD_qubits)
        # else:
        #     for qubit in range(cir.num_qubits):
        #         new_map[qubit][1] += 1
        maps.append(new_map)
        new_map = copy.deepcopy(maps[-1])
        
    return maps
maps = get_maps(cir)
maps

[[[1, 1], [1, 2], [1, 3], [1, 4], [1, 5]],
 [[1, 2], [1, 2], [1, 3], [1, 4], [1, 5]],
 [[1, 3], [1, 2], [1, 3], [1, 4], [1, 5]],
 [[1, 4], [1, 3], [1, 3], [1, 4], [1, 5]],
 [[1, 5], [1, 4], [1, 3], [1, 4], [1, 5]],
 [[1, 6], [1, 5], [1, 4], [1, 4], [1, 5]],
 [[1, 7], [1, 6], [1, 5], [1, 4], [1, 5]],
 [[1, 8], [1, 7], [1, 6], [1, 5], [1, 5]]]

In [7]:
from Enola.route import *

In [8]:
route = QuantumRouter(num_q,maps,cir.gate_list,[2,2*num_q+1])
route.run()

In [9]:
route.movement_list

[[[[0, (1, 1), (1, 2)]]],
 [[[0, (1, 2), (1, 3)]]],
 [[[1, (1, 2), (1, 3)], [0, (1, 3), (1, 4)]]],
 [[[1, (1, 3), (1, 4)], [0, (1, 4), (1, 5)]]],
 [[[1, (1, 4), (1, 5)], [2, (1, 3), (1, 4)], [0, (1, 5), (1, 6)]]],
 [[[1, (1, 5), (1, 6)], [2, (1, 4), (1, 5)], [0, (1, 6), (1, 7)]]],
 [[[3, (1, 4), (1, 5)],
   [1, (1, 6), (1, 7)],
   [0, (1, 7), (1, 8)],
   [2, (1, 5), (1, 6)]]]]

In [10]:
total_paralled = cir.gate_list
all_movements = []
for num in range(len(maps) - 1):
    for paral_moves in route.movement_list[num]:
        all_movements.append(paral_moves)
gate_num = cir.num_gate
t_idle, Fidelity, move_fidelity = DasAtom_fidelity(total_paralled, all_movements, num_q, gate_num)
print(["Fidelity:", Fidelity])
print(["t_idle:", t_idle])
print(["move_fidelity", move_fidelity])
print(["Movement times", len(all_movements)])
print(["parallel times", len(total_paralled)])

['Fidelity:', 0.9028061313935791]
['t_idle:', 2994.9090909090914]
['move_fidelity', 0.9996012916265293]
['Movement times', 7]
['parallel times', 8]


In [11]:
Enola_fidelity(total_paralled, all_movements, num_q, gate_num)

{'cir_fidelity_1q_gate': 1,
 'cir_fidelity_2q_gate': 0.9046104802746175,
 'cir_fidelity_2q_gate_for_idle': 0.9276563692349433,
 'cir_fidelity_atom_transfer': 0.9193926150309792,
 'cir_fidelity_coherence': 0.998058231422544,
 'cir_fidelity': 0.7700264398244803}

In [12]:
data_list = []
for n in range(5,51):
    cir = QFT(n)
    qasm2.dump(cir.circuit,f"Bench/qft_{n}.qasm")
    maps = get_maps(cir)
    route = QuantumRouter(cir.num_qubits,maps,cir.gate_list,[2,2*n+1])
    route.run()
    total_paralled = cir.gate_list
    all_movements = []
    for num in range(len(maps) - 1):
        for paral_moves in route.movement_list[num]:
            all_movements.append(paral_moves)
    gate_num = cir.num_gate
    # t_idle, Fidelity, move_fidelity = DasAtom_fidelity(total_paralled, all_movements, n, gate_num)
    # data = {"bench file": f"qft_{n}.qasm",
    #         "cir fidelity": Fidelity,
    #         "idle time": t_idle,
    #         "move fidelity": move_fidelity}
    data = Enola_fidelity(total_paralled, all_movements, n, gate_num)
    data_list.append(data)

In [13]:
data_pd = pd.DataFrame(data_list)
data_pd.to_csv("qft.csv")